In [8]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

from tensorflow.keras import layers, Model
from tensorflow.keras import datasets

from tensorflow.keras import losses
from tensorflow.keras import optimizers

from tensorflow.keras import callbacks

In [9]:
def define_discriminator(in_shape=(32,32,3)):
	model = keras.models.Sequential()
	
	model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=in_shape)) #16x16x128
	model.add(layers.LeakyReLU(alpha=0.2))
	
	model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same')) #8x8x128
	model.add(layers.LeakyReLU(alpha=0.2))
	
	model.add(layers.Flatten()) #shape of 8192
	model.add(layers.Dropout(0.4))
	model.add(layers.Dense(1, activation='sigmoid')) #shape of 1
	# compile model
	opt = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [10]:
def define_generator(latent_dim):    #latent_dim is the dimension of the latent vector (e.g., 100)
	model = keras.models.Sequential()
	# We will reshape input latent vector into 8x8 image as a starting point. 
    #So n_nodes for the Dense layer can be 128x8x8 so when we reshape the output 
    #it would be 8x8x128 and that can be slowly upscaled to 32x32 image for output.
	n_nodes = 128 * 8 * 8  #8192 nodes
	model.add(layers.Dense(n_nodes, input_dim=latent_dim)) #Dense layer so we can work with 1D latent vector
	model.add(layers.LeakyReLU(alpha=0.2))
	model.add(layers.Reshape((8, 8, 128)))  #8x8x128 dataset from the latent vector. 
	# upsample to 16x16
	model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')) #16x16x128
	model.add(layers.LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')) #32x32x128
	model.add(layers.LeakyReLU(alpha=0.2))
	# generate
	model.add(layers.Conv2D(3, (8,8), activation='tanh', padding='same')) #32x32x3
	return model

In [11]:
def define_gan(generator, discriminator):
	discriminator.trainable = False  #Discriminator is trained separately. So set to not trainable.
	# connect generator and discriminator
	model = keras.models.Sequential()
	model.add(generator)
	model.add(discriminator)
	# compile model
	opt = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model


# load cifar training images
def load_real_samples():
	(trainX, _), (_, _) = datasets.cifar10.load_data()
	# cConvert to float and scale.
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5  #Generator uses tanh activation so rescale 
                            #original images to -1 to 1 to match the output of generator.
	return X

def generate_real_samples(dataset, n_samples):
	# choose random images
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# select the random images and assign it to X
	X = dataset[ix]
	# generate class labels and assign to y
	y = np.ones((n_samples, 1)) ##Label=1 indicating they are real
	return X, y

# generate n_samples number of latent vectors as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict using generator to generate fake samples. 
	X = generator.predict(x_input)
	# Class labels will be 0 as these samples are fake. 
	y = np.zeros((n_samples, 1))  #Label=0 indicating they are fake
	return X, y

In [12]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)  #the discriminator model is updated for a half batch of real samples 
                            #and a half batch of fake samples, combined a single batch. 
	# manually enumerate epochs and bacthes. 
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
            
            # Train the discriminator on real and fake images, separately (half batch each)
        #Research showed that separate training is more effective. 
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
            ##train_on_batch allows you to update weights based on a collection 
            #of samples you provide
            #Let us just capture loss and ignore accuracy value (2nd output below)
			d_loss_real, _ = d_model.train_on_batch(X_real, y_real) 
			
            # generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss_fake, _ = d_model.train_on_batch(X_fake, y_fake)
            
            #d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) #Average loss if you want to report single..
            
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
             
            # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the generator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)			
			y_gan = np.ones((n_batch, 1))
            
            # Generator is part of combined model where it got directly linked with the discriminator
        # Train the generator with latent_dim as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			
            # Print losses on this batch
			print('Epoch>%d, Batch %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss_real, d_loss_fake, g_loss))
	# save the generator model
	g_model.save('cifar_generator_2epochs.h5')

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=20)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
I0000 00:00:1711788522.981762      82 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step  
Epoch>1, Batch 1/390, d1=0.732, d2=0.712 g=0.694
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


W0000 00:00:1711788534.345424      82 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Epoch>1, Batch 2/390, d1=0.721, d2=0.715 g=0.692
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 3/390, d1=0.719, d2=0.716 g=0.691
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 4/390, d1=0.718, d2=0.717 g=0.688
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 5/390, d1=0.720, d2=0.719 g=0.686
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 6/390, d1=0.721, d2=0.721 g=0.683
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 7/390, d1=0.723, d2=0.723 g=0.680
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 8/390, d1=0.724, d2=0.725 g=0.676
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 9/390, d1=0.726, d2=0.728 g=0.672
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 10/390, d1=0.728, d2=0.730 g=0.666
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 11/390, d1=0.732, d2=0.734 g=0.660
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 12/390, d1=0.735, d2=0.739 g=0.654
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch>1, Batch 13/390, d1=0.739, d2=0.74

In [ ]:
# Plot generated images 
def show_plot(examples, n):
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :, :])
	plt.show()

# load model
model = keras.models.load_model('cifar_generator_250epochs.h5') #Model trained for 100 epochs
# generate images
latent_points = generate_latent_points(100, 25)  #Latent dim and n_samples
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0

X = (X*255).astype(np.uint8)

# plot the result
show_plot(X, 5)